## Before running, pip install numba, pyglet and nbody

# 2-D simulation using numba and pyglet

In [7]:
#This simulation also contains collisions
import itertools
import math
import pyglet
import pyglet.gl as gl
from random import random
from numba import jit
import time


window = pyglet.window.Window(1400, 800)


def make_circle_fill():
    """ Adds a vertex list of circle polygon to batch and returns it. """
    num_points = 40
    batch = pyglet.graphics.Batch()
    rad = math.pi * 2 / num_points # getting 360 / n in radians
    index = list(itertools.chain.from_iterable( (0, x-1, x)  for x in range(2, num_points+1) ))
    index += [0, 1, num_points] # end of fan
    vertices = [0, 0] # adding center of fan
    for i in range(1, num_points + 1):
        angle = rad * i
        vertices += [math.cos(angle), math.sin(angle)]
    vertices += [1, 0] # adding end of fan
    circle = pyglet.graphics.vertex_list_indexed(num_points+2, index, ('v2f', vertices))
    return circle


@jit(nopython=True, cache=True)
def compute_radius(mass):
    return mass**0.333 * 0.4


# Initialize stuff
circlefill = make_circle_fill()

total_objects = 1000

masses = []
radii = []
px = []
py = []
vx = []
vy = []
deleted = []
for i in range(total_objects):
    mass = random() * 100
    masses.append(mass)
    radii.append(compute_radius(mass))
    px.append(random() * 1400)
    py.append(random() * 800)
    vx.append((random() - 0.5) * 4)
    vy.append((random() - 0.5) * 4)
    deleted.append(0)


def timer(dt):
    global total_objects
    total_objects = turn(total_objects, masses, radii, px, py, vx, vy, deleted)


@jit(parallel = True, nopython=True, cache=True)
def turn(total_objects, masses, radii, px, py, vx, vy, deleted):
    to_clean = set()
#     seconds = time.time()
    for i in range(total_objects):
        for j in range(i + 1, total_objects):
            # distance on each axis
            dx = px[i] - px[j]
            dy = py[i] - py[j]

            # square distances
            dx2 = dx ** 2
            dy2 = dy ** 2
            r2 = dx2 + dy2
            # Used to translate acceleration value into a vector later on
            normalizer = abs(dx) + abs(dy)

            # check collision
            # Objects that are already collided with something else can't collide again
            if deleted[j] == 0 and dx2 + dy2 < (radii[i] + radii[j]) ** 2:
                # j will be removed, i will gain all of j's mass and momentum
                total_mass = masses[i] + masses[j]
                # new coordinates
                px[i] = (px[i] * masses[i] + px[j] * masses[j]) / total_mass
                py[i] = (py[i] * masses[i] + py[j] * masses[j]) / total_mass
                # new velocity based on momentum
                vx[i] = (vx[i] * masses[i] + vx[j] * masses[j]) / total_mass
                vy[i] = (vy[i] * masses[i] + vy[j] * masses[j]) / total_mass

                radii[i] = compute_radius(total_mass)
                masses[i] = total_mass
                # Cleanup j after all of this is over (until then we need to compute its influence on other objects)
                to_clean.add(j)
                deleted[j] = True
            # No collision? Compute interaction
            else:
                vx[i] -= dx * normalizer / r2 * masses[j] / 1000000.0
                vy[i] -= dy * normalizer / r2 * masses[j] / 1000000.0
                vx[j] += dx * normalizer / r2 * masses[i] / 1000000.0
                vy[j] += dy * normalizer / r2 * masses[i] / 1000000.0

        px[i] += vx[i]
        py[i] += vy[i]
#     print("Seconds since beginning", seconds)

    # Clean up deleted objects after the round completion, in reverse order
    for i in sorted(to_clean)[::-1]:
        masses.pop(i)
        radii.pop(i)
        px.pop(i)
        py.pop(i)
        vx.pop(i)
        vy.pop(i)
        deleted.pop(i)
        total_objects -= 1
    return total_objects


@window.event
def on_draw():
    global total_objects
#     total_objects = turn(total_objects, masses, radii, px, py, vx, vy)
    print(total_objects)
    gl.glClear(gl.GL_COLOR_BUFFER_BIT)
    for i in range(total_objects):
        gl.glPushMatrix()
        gl.glColor3f(1,1,0)
        gl.glTranslatef(px[i], py[i], 0)
        gl.glScalef(radii[i], radii[i], 1)
        circlefill.draw(gl.GL_TRIANGLES)
        gl.glPopMatrix()

# Schedule an update every 1/30th of a second. After each update, on_draw() is called automatically
pyglet.clock.schedule_interval(timer, 1/30.0)
pyglet.app.run()

1000


C:\Users\manavgakhar\anaconda3\lib\site-packages\numba\core\typed_passes.py:313: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see https://numba.pydata.org/numba-doc/latest/user/parallel.html#diagnostics for help.

File "<ipython-input-7-cc2a267a2de4>", line 63:
@jit(parallel = True, nopython=True, cache=True)
def turn(total_objects, masses, radii, px, py, vx, vy, deleted):
^

  warnings.warn(errors.NumbaPerformanceWarning(msg,
C:\Users\manavgakhar\anaconda3\lib\site-packages\numba\core\ir_utils.py:2031: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'deleted' of function 'turn'.

For more information visit https://numba.pydata.org/numba-doc/latest/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "<ipython-input-7-cc2a26

988
977
972
971
964
959
953
945
940
934
933
929
926
917
916
911
908
903
901
899
893
887
882
877
872
872
869
864
859
858
855
854
852
847
845
840
835
831
824
820
818
814
811
804
803
799
795
793
792
788
785
783
780
775
771
765
762
755
753
751
751
747
747
746
742
740
736
729
725
721
716
716
713
711
707
705
700
696
689
683
681
679
673
667
661
657
654
650
647
645
642
638
633
628
625
617
614
608
603
599
597
590
589
582
576
571
563
561
558
548
532
527
521
517
511
507
498
490
483
480
477
467
458
453
451
441
432
430
422
417
409
405
401
397
386
376
371
356
349
342
336
327
318
313
307
302
299
289
284
275
269
262
257
251
245
239
227
226
216
211
204
198
192
184
179
175
168
162
156
148
143
139
132
124
121
118
116
114
113
113
110
106
101
101
98
95
94
94
94
93
90
88
87
87
87
83
82
81
81
81
81
80
80
78
78
76
76
76
76
76
76
76
75
75
75
74
74
74
73
73
73
73
73
73
72
71
70
69
69
69
69
69
69
69
69
69
69
69
69
69
69
68
68
68
68
68
68
68
67
67
67
67
67
67
67
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66


# 10 Particle 3-D simulation

In [ ]:
from nbody import rand, animate, save


filename = "particles_3D"

# Initializing a random system() 
L = rand(10, x0 = (0,10), p = 3, m = (1E9, 1E3))

T = 5E-2
dt = 1E-4

# Solving for the given T and dt
L.solve(T, dt, collision = True)


save(L, filename)

# Displaying an animation of the system
animate(L)

<IPython.core.display.Javascript object>


SIMULATION INFO:

	Particles		10
	Dimensions		3
	T			0.05
	dt			0.0001
	Steps			500
	CUDA			Inactive
	Collisions		Active

	Status			Complete – Total Time Elapsed 00:00:01


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>